# **CNN with mnist using subclassing API**

In [4]:
# import TensorFlow
import tensorflow as tf
print(tf.__version__) # find the version number (should be 2.XX)

# 그래픽카드 유무 확인 및 메모리 확장 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    print('사용가능한 GPU 갯수 : ', len(gpus), '\n')

    try:
        # 프로그램이 실행되어 더많은 GPU 메모리가 필요하면 , 텐서플로 프로세스에 할당된 GPU메모리
        # 영역을 확장할 수 있도록 허용
        tf.config.experimental.set_memory_growth(gpus[0], True)

    except RuntimeError as e:
        # 프로그램 시작시에 접근 가능한 장치가 설정되어야만 함
        print(e)

# 설치된 GPU 상세내용 확인
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())                          

2.6.0
사용가능한 GPU 갯수 :  1 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14580967943434237972
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11345264640
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16116302685718589885
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


# **tf,data 사용법**

TensorFlow에서는 Dataset 이라는 built-in-API를 제공하고 있어서 위의 작업을 쉽게 처리할 수 있다.<br> 이 포스트에서는 입력 파이프라인을 만들어서 모델에 데이터를 효율적으로 공급하는 방법을 살펴볼 것이다.<br> 또한, 흔하게 볼 수 있는 예시를 다루면서 Dataset의 기본적인 메커니즘을 설명할 것이다.

Dataset을 사용하려면 세 가지 단계를 거쳐야한다.

* 데이터셋 생성하기: 사용하려는 데이터로부터 Dataset 인스턴스를 만든다.
* Iterator(반복자) 생성하기. 생성된 데이터를 사용해서 Iterator 인스턴스를 만들어 Dataset을 iterate시킨다.
* 데이터 사용하기. 생성된 iterator를 사용해서 모델에 공급할 dataset으로부터 요소를 가져올 수 있다.
---

# 데이터 불러오기/Iterator(반복자) 생성하기

일단 dataset안에 넣을 데이터가 필요하다.

* list
* numpy array
* tensor

from_tensor_slices()

요소가 지정된 텐서의 슬라이스인 데이터 집합을 만듭니다.

---

## tf.data.Dataset.from_tensor_slices()

주어진 텐서는 첫 번째 치수를 따라 슬라이스됩니다.<br> 이 작업은 입력 텐서의 구조를 보존하여 각 텐서의 첫 번째 치수를 제거하고 데이터 집합 차원으로 사용합니다.<br> 모든 입력 텐서는 첫 번째 차원에서 크기가 동일해야 합니다.

---



In [56]:
# list 불러오기
ds = tf.data.Dataset.from_tensor_slices([[1, 2, 3, 4, 5 ],
                                         [5, 6, 7, 8, 9]])
ds

<TensorSliceDataset shapes: (5,), types: tf.int32>

In [57]:
import numpy as np
# numpy에서 불러오기
# create a random vector of shape(10, 2)
x = np.random.sample((10, 2))

# make a dataset from a numpy array
ds = tf.data.Dataset.from_tensor_slices(x)
print(type(ds))
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


[array([0.69797648, 0.84345139]),
 array([0.36404385, 0.35480772]),
 array([0.16231554, 0.34205108]),
 array([0.47672615, 0.03579508]),
 array([0.93976285, 0.16825536]),
 array([0.18710618, 0.53596571]),
 array([0.91306769, 0.53159284]),
 array([0.59621266, 0.32854768]),
 array([0.72749162, 0.12646779]),
 array([0.08756933, 0.65446523])]

또한 데이터를 특성(feature)과 라벨(label)로 나누어 사용하는 경우처럼, 한 개 이상의 numpy 배열을 넣을 수도 있다.

In [58]:
features, labels = (np.random.sample((10, 2)), np.random.sample((10, 1)))
ds = tf.data.Dataset.from_tensor_slices((features, labels))

print(type(ds))
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


[(array([0.66305057, 0.77718113]), array([0.45018888])),
 (array([0.66644377, 0.02327975]), array([0.04310358])),
 (array([0.25624371, 0.70470604]), array([0.78552806])),
 (array([0.23089809, 0.25771667]), array([0.18891595])),
 (array([0.21098424, 0.227012  ]), array([0.35470743])),
 (array([0.88631135, 0.05479445]), array([0.76394321])),
 (array([0.10803926, 0.3808018 ]), array([0.1142942])),
 (array([0.19108316, 0.63481404]), array([0.13047112])),
 (array([0.23967793, 0.58823585]), array([0.64288857])),
 (array([0.63545697, 0.18012575]), array([0.08490395]))]

In [59]:
# tensor에서 불러오기
ds = tf.data.Dataset.from_tensor_slices(tf.random.uniform([100, 2]))
ds

<TensorSliceDataset shapes: (2,), types: tf.float32>

## tf.random.uniform()

tf.random.uniform은 원하는 형태의 랜덤 값을 가진 배열을 만듭니다.

In [60]:
# shape 지정
tf.random.set_seed(1)
result = tf.random.uniform(shape=[5, 5])
result

# 랜덤 값의 최소범위 (minval)
result = tf.random.uniform(shape=[5, 5], minval=-10)
result

# 랜덤 값의 최대범위 (maxval)
result = tf.random.uniform(shape=[5, 5], minval=-10, maxval=10)
result

# dtype 형식 지정
result = tf.random.uniform(shape=[5, 5], minval=-10, maxval=10, dtype=tf.int32)
result

# name에는 글자가 들어갑니다. 지금은 안 보이지만, 나중에 그래프를 그리거나 할 때 참고될 수 있습니다.
result = tf.random.uniform(shape=[5, 5], minval=-10, maxval=10, dtype=tf.int32, name='uniform')
result

<tf.Tensor: shape=(5, 5), dtype=int32, numpy=
array([[ 4,  4,  9,  2,  1],
       [ 0,  7,  8, -7,  1],
       [ 3,  7, -1,  1,  4],
       [-1,  1,  7,  9,  1],
       [-5, -4, -7,  8, -4]], dtype=int32)>

## .as_numpy_iterator()

데이터 집합의 요소에 걸쳐 텐서가 numpy 배열로 변환된 테이블을 반환합니다.

In [61]:
for num in ds.as_numpy_iterator():
    print(num)

[0.33260047 0.64583516]
[0.69990575 0.26295936]
[0.5514865  0.67170334]
[0.02142632 0.17200863]
[0.0739969 0.7926762]
[0.84816945 0.7786716 ]
[0.6296967 0.8350655]
[0.2947855  0.42360163]
[0.68189573 0.19928575]
[0.8069899 0.8871764]
[0.8765991  0.18183112]
[0.50295806 0.02347124]
[0.14948988 0.0427047 ]
[0.3609544 0.7918347]
[0.7131158 0.8788419]
[0.68504405 0.24746811]
[0.30284965 0.8154969 ]
[0.08793283 0.7530719 ]
[0.06086636 0.7210591 ]
[0.1629262 0.7354362]
[0.5063546 0.2863058]
[0.71820974 0.8625773 ]
[0.37951863 0.77473783]
[0.83857393 0.81271625]
[0.39438844 0.53415525]
[0.6800957  0.22501433]
[0.7166109  0.50490963]
[0.7981851  0.56560564]
[0.6342473  0.41699862]
[0.13728333 0.47109532]
[0.38881063 0.68199575]
[0.4168657 0.8980521]
[0.20606661 0.13203704]
[0.8918228  0.21598113]
[0.7790537  0.27799726]
[0.53905344 0.46288633]
[0.948316   0.22863638]
[0.17012084 0.77538586]
[0.4048772  0.13249528]
[0.71531856 0.40542364]
[0.27091193 0.57860374]
[0.47161138 0.7239896 ]
[0.84486

## .batch():

이 데이터 집합의 연속 요소를 배치로 결합합니다.<br>
batch(batch_size, drop_remainder=False)

In [62]:
ds = tf.data.Dataset.range(10)
# 데이터 배치의 크기를 정함
ds = ds.batch(3)
list(ds.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7, 8]), array([9])]

In [63]:
ds = tf.data.Dataset.range(10)
# 배치를 시키고 남은 데이터를 drop 할 것인지 여부 (drop_remainder)
ds = ds.batch(3, drop_remainder=True)
list(ds.as_numpy_iterator())

[array([0, 1, 2]), array([3, 4, 5]), array([6, 7, 8])]

## .filter()

* 조건에 따라 이 데이터 집합을 필터링합니다.
* 매개변수에는 tensor 자료형이 들어가야함

In [64]:
ds = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
ds = ds.filter(lambda x: x < 3)
list(ds.as_numpy_iterator())

[1, 2]

In [79]:
def filter_fn(x):
    # less() 는 x < y의 요소별 진리값을 찾는 데 사용
    return tf.math.less(x, 3)

ds = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5])
print(type(ds))
ds = ds.filter(filter_fn)
list(ds.as_numpy_iterator())

<class 'tensorflow.python.data.ops.dataset_ops.TensorSliceDataset'>


[1, 2]

## .apply()

* 변환 함수를 이 데이터 집합에 적용합니다.
* Dataset인수 를 취하고 변환된 을 반환하는 함수로 표현되는 사용자 정의 변환을 연

In [81]:
ds = tf.data.Dataset.range(100)
def dataset_fn(ds):
    return ds.filter(lambda x: x <10)

ds = ds.apply(dataset_fn)
list(ds.as_numpy_iterator())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

## .shuffle()

* 이 데이터 집합의 요소를 임의로 섞습니다.
* 이 데이터 집합은 버퍼를 buffer_size 요소로 채운 다음 이 버퍼에서 임의로 요소를 샘플링하여 선택한 요소를 새 요소로 바꿉니다.
* 완벽한 쉐이핑을 위해서는 데이터 집합의 전체 크기보다 크거나 같은 버퍼 크기가 필요합니다.

In [130]:
ds = tf.data.Dataset.range(3)
# 셔플 순서가 각 에포크에 대해 달라야 하는지 여부(reshufflw_each_iteration)
ds = ds.shuffle(3, reshuffle_each_iteration=True)
                    # 셔플링을 위해서는 데이터 세트의 전체 크기보다 크거나 같은 버퍼 크기가 필요
# 반복횟수
ds = ds.repeat(2)
# [1, 0, 2, 1, 2, 0]
print(list(ds.as_numpy_iterator()))
#################################################################

ds = tf.data.Dataset.range(3)
ds = ds.shuffle(3, reshuffle_each_iteration=False)

# 반복횟수
ds = ds.repeat(2)
# [1, 0, 2, 1, 0, 2]
print(list(ds.as_numpy_iterator()))

[2, 0, 1, 2, 1, 0]
[1, 0, 2, 1, 0, 2]


# **CNN with mnist using subclassing API**


MNIST 데이터셋을 로드하여 준비합니다

In [131]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train /255.0, x_test / 255.0

# 채널 차원을 추가합니다. (...은 기존배열을 나타냄)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11501568/11490434 [==============================] - 0s 0us/step


tf.data를 사용하여 데이터셋을 섞고 배치를 만듭니다:

In [141]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

print(train_ds)
print(test_ds)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>
<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>


In [148]:
# 파이썬 클래스와 상속
from builtins import object, super

class Animal(object):

    def __init__(self, genus):
        self.genus = genus

class Dog(Animal):
    def __init__(self, genus):
        super().__init__(genus)

class Dog2(Animal):
    def __init__(self, genus):
        super(Dog2, self).__init__(genus)

# super는 부모객체를 나타냄,super() , super(ClassName, self)나 똑같음 버젼차이
x = Dog('Canis')
y = Dog2('Canis')
print(x.genus)
print(y.genus)

Canis
Canis


# **모델 서브클래싱(subclassing) API**

케라스(Keras)의 모델 서브클래싱(subclassing) API를 사용하여 tf.keras 모델을 만듭니다:

In [168]:
class MyModel(Model):
    # 생성되면 자동으로 실행되는 __init__생성자
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

훈련에 필요한 옵티마이저(optimizer)와 손실 함수를 선택합니다:

In [169]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()


모델의 손실과 성능을 측정할 지표를 선택합니다. 에포크가 진행되는 동안 수집된 측정 지표를 바탕으로 최종 결과를 출력합니다.

In [170]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
# 예측이 정수 레이블과 일치하는 빈도를 계산합니다.

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

tf.GradientTape를 사용하여 모델을 훈련합니다:

In [171]:
# 즉시 실행모드, 자동으로 그래프를 생성
# tf1.x 형태처럼 그래프 생성과 실행이 분리된 형태로 해당 함수내의 로직이 실행 (성능 향상)
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
    # 중간 연산 과정(함수, 연산)을 테이프(tape)에 차곡차곡 기록
        predictions = model(images)
        loss = loss_object(labels, predictions)
    # 경사하강법 (자동 미분)
    gradients = tape.gradient(loss, model.trainable_variables)
    # 처리 된 그라디언트를 적용하도록 요청
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

In [172]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [173]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.13547536730766296, 정확도: 96.02833557128906, 테스트 손실: 0.06125630438327789, 테스트 정확도: 97.88999938964844
에포크: 2, 손실: 0.08850956708192825, 정확도: 97.35833740234375, 테스트 손실: 0.0585399828851223, 테스트 정확도: 98.0250015258789
에포크: 3, 손실: 0.06608036160469055, 정확도: 97.99722290039062, 테스트 손실: 0.058740317821502686, 테스트 정확도: 98.09667205810547
에포크: 4, 손실: 0.05281568318605423, 정확도: 98.39083862304688, 테스트 손실: 0.05691465735435486, 테스트 정확도: 98.21500396728516
에포크: 5, 손실: 0.04404117166996002, 정확도: 98.65033721923828, 테스트 손실: 0.05704478919506073, 테스트 정확도: 98.28399658203125
에포크: 6, 손실: 0.03801508620381355, 정확도: 98.83139038085938, 테스트 손실: 0.058126457035541534, 테스트 정확도: 98.29166412353516
에포크: 7, 손실: 0.033370908349752426, 정확도: 98.9721450805664, 테스트 손실: 0.05887372046709061, 테스트 정확도: 98.32142639160156
에포크: 8, 손실: 0.029731670394539833, 정확도: 99.08333587646484, 테스트 손실: 0.061245959252119064, 테스트 정확도: 98.33499908447266
에포크: 9, 손실: 0.026980239897966385, 정확도: 99.16796112060547, 테스트 손실: 0.06327974051237106, 테스트 정확도


# **@tf.function으로 성능 향상하기**

@tf.fuctnion을 이해하기 위해서는 tensorflow 개발과정의 역사(?)를 조금 이해하셔야하는데요. tf 1.x 버전대에서는 그래프의 생성과 실행을 분리하고 값을 실행할때는 Session이라는 것을 열어서 값을 실행하는 형태였는데요. 이렇게 진행하다보니 값을 계산하고 싶을때마다 Session을 이용해서 실행을 해주어야만해서 프로그래밍 과정상에 불편함이 많았습니다. 따라서 tf 2.x 버전대에서는 Session을 삭제하고 바로 값을 실행할 수 있는 Eager Execution이라는 것이 적용되었는데요. 따라서 값을 계산할때 별도의 Session을 열지 않고도 편리하게 진행할 수 있게 되었습니다. 그럼 왜 굳이 tf 1.x대에서는 저렇게 복잡하게 그래프의 생성과 실행을 분리했느냐라고 생각해 볼 수 있는데요. 해당 형태가 성능상(=속도)의 이점이 있기 때문입니다.

여기서 @tf.function 관련된 내용이 나오게 되는데요. def 위에 @tf.fucnction을 붙이면 마치 tf2.x 버전에서도 tf1.x 형태처럼 그래프 생성과 실행이 분리된 형태로 해당 함수내의 로직이 실행되게 됩니다. 따라서 상황에 따라서 성능이 약간 향상 될 수 있는데요.(=실행 속도가 약간 빨라질 수 있습니다.) 다만 해당 annoation을 붙이면 tf1.x처럼 해당 함수내의 값을 바로 계산해볼수 없어서 디버깅이 불편해질수 있습니다. 따라서 모든 로직에 대한 프로그래밍이 끝난 상태에서 @tf.fuction을 붙이는 것이 좋습니다.

즉 정리하면

* @tf.fucntion을 붙이면 tf1.x 스타일로 해당 함수내의 로직이 동작한다.
* 따라서 상황에 따라 속도가 약간 빨라질 수 있다.
* 다만 해당 annotation을 붙이면 값을 바로 계산해볼수 없어서 모든 로직에 대한 프로그래밍이 끝난 뒤에 붙이는 것이 좋다.

In [174]:
@tf.function
def add(a, b):
    print(a, b)
    # print('tf.variable 1 =',tf.Variable(3))
    return a + b

print('tf.variable 2 = ', tf.Variable(3))
add(tf.ones([2, 2]), tf.ones([2, 2])) # [[2., 2.], [2., 2.]]

tf.variable 2 =  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=3>
Tensor("a:0", shape=(2, 2), dtype=float32) Tensor("b:0", shape=(2, 2), dtype=float32)


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [176]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

<tf.Variable 'Variable:0' shape=() dtype=float32> 1.0


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>